Outline:
* Read in all data to a dataframe called df
* Separate categorical data in a dataframe called df_categorical, remove categorical features from df
* Reconcile some variables that are falsely interpreted as categorical with df
    * We do this by using a temporary dataframe (df_storage), cleaning the data, then rejoining it with df
* One-hot encode the data in df_categorical and store it in a new dataframe called df_ohe, then recombine with df
* Leverage U.S. Census Gazetteer Files and a government-maintained list of Medicare hospitals in order to engineer a feature called 'hosp_per_sq_mi', a measure of the density of Medicare hospitals in the zip code of each member
* Import a list of reduced variables to create the final modified training set

Note: Dataframes are not displayed (except Gazetteer and Medicare hospital info) for data privacy reasons.

Importing libraries:

In [202]:
import pandas as pd
import numpy as np


# Data cleaning and one-hot encoding

Importing data:

In [203]:
# import training set
df = pd.read_csv("2020_Competition_Training.csv")

# output number of features
print(f'{len(df)} total samples')
print(f'{len(df.columns)} total features')


C:\Users\HP\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (80,193) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


69572 total samples
826 total features


Separating categorical variables from numeric variables:

In [204]:
# separate categorical data
df_categorical = df.select_dtypes(exclude=['number','bool_'])

# output amount of categorical variables
print(f'{len(df_categorical.columns)} categorical variables')

# remove categorical features from initial df, but retain zip code (we'll use this to merge later)
df.drop(list(df_categorical.columns), axis=1, inplace=True)
df['zip_cd'] = df_categorical['zip_cd']


22 categorical variables


Separate features that were falsely interpreted as categorical:

In [205]:
# this row falsely interpreted as categorical due to numerical strings (same with zip code)
# store it in a temporary df which we will rejoin with main df after cleaning
df_storage = df_categorical[['cons_cmys']].copy()

# drop this column since essentially all nan
print(f'{df_categorical["hedis_ami"].isnull().sum()} nan values in "hedis_ami" column')

# remove both of these from categorical data
df_categorical.drop(['hedis_ami', 'cons_cmys'], axis=1, inplace=True)


69339 nan values in "hedis_ami" column


C:\Users\HP\anaconda3\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Cleaning missing data (filled by asterisks, replace these with nan) in storage df

In [206]:
df_storage['cons_cmys'].replace('*', np.nan, inplace=True)


Sending numeric data to df_storage after converting from strings to floats, recombining cleaned numeric strings with main df

In [207]:
df_storage['cons_cmys'] = df_storage['cons_cmys'].astype('float64')

df = df.join(df_storage)


Converting Y/N indicators to 1/0 indicators (this happens in the 'hedis' features)

In [208]:
# store names of 'hedis' columns for later cleaning
hedis_cols = []

# iterate through columns, append 'hedis' columns to list
for col in df_categorical.columns:
    if str(col)[0:5] == 'hedis':
        hedis_cols.append(col)
        
        # convert from Y/N to binary indicators (1/0), making categorical data numeric
        df_categorical[col] = df_categorical[col].map({'Y': 1, 'N': 0})


Separating 'hedis' features into a new dataframe since they are no longer categorical and thus we don't want to one-hot encode them

In [209]:
# initialize new dataframe for hedis variables
df_hedis = df_categorical[hedis_cols].copy()

# remove hedis variables from df_categorical
df_categorical.drop(columns = hedis_cols, inplace=True)


Recombining hedis variables with main df since no longer categorical

In [210]:
df = df.join(df_hedis)


Removing IDs from df_categorical, placing back into df

In [211]:
 # store ids for later use
df_ids = df_categorical[['person_id_syn']].copy()

# remove ids since don't want to one-hot encode them
df_categorical.drop(columns=['person_id_syn'], inplace=True)

# recombine ids with main df
df = df.join(df_ids)


Some quick formatting:

In [212]:
# move ID column to front, rename
df.insert(0, 'ID', df['person_id_syn'])

# drop old ID column
df.drop(columns = ['person_id_syn'], inplace = True)


One-hot encoding:

In [213]:
# store column names to loop through later
cols_to_ohe = [col for col in df_categorical]

# initialize df_categorical with one-hot encoded version of first variable
df_ohe = pd.get_dummies(df_categorical.src_platform_cd, prefix='src_plat')


In [214]:
# don't want to one-hot encode zip code, county code, and state
geographic_info = cols_to_ohe[-3:]

# store columns that we actually want one-hot encode
cols_ohe = cols_to_ohe[:-3]


In [215]:
# looping through, skipping first variable ('src_platform_cd') since we already one-hot encoded it to initialize df_ohe
for col in cols_ohe[1:]:

    # use temporary dataframe to store one-hot encoded version of feature
    this_df = pd.get_dummies(df_categorical[col], prefix = col)
    
    # join with all other one-hot encoded features
    df_ohe = df_ohe.join(this_df)


In [216]:
# dropping one-hot encoded columns that arise from missing data
df_ohe.drop(columns = ['cms_ra_factor_type_cd_*'], inplace = True)


In [217]:
# joining one-hot encoded data with main df
df = df.join(df_ohe)


# Creating 'hosp_per_sq_mi' feature:

Read in zip code data:

In [218]:
df_zip = pd.read_csv("2019_Gaz_zcta_national.txt", delimiter = '\t')
df_zip


,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,601,166659747,799292,64.348,0.309,18.180555,-66.749961
1,602,79307534,4428428,30.621,1.710,18.361945,-67.175597
2,603,81887188,181412,31.617,0.070,18.455183,-67.119887
3,606,109579993,12487,42.309,0.005,18.158327,-66.932928
4,610,93013428,4172059,35.913,1.611,18.294032,-67.127156
...,...,...,...,...,...,...,...
33139,99923,42162326,2117,16.279,0.001,56.000518,-130.037474
33140,99925,144071036,34333408,55.626,13.256,55.550203,-132.945947
33141,99926,343944587,292859014,132.798,113.074,55.138352,-131.470425
33142,99927,589690080,18001568,227.681,6.950,56.239062,-133.457924


Some data cleaning on zip code data:

In [219]:
# convert GEOID to strings so we can merge on zip code
df_zip['GEOID'] = df_zip['GEOID'].astype(str)

# rename columns of zip code data
df_zip = df_zip.rename(columns = {"GEOID": "zip_cd", "ALAND_SQMI" : "zip_sq_mi"})


Merging zip code data with categorical data:

In [220]:
df_categorical = df_categorical.merge(df_zip, how = 'left', on = 'zip_cd')


Dropping unnecessary columns:

In [221]:
df_categorical = df_categorical.drop(columns = ['ALAND', 'AWATER', 'AWATER_SQMI', 'INTPTLAT'])


Read in Medicare hospital data:

In [222]:
df_medicare = pd.read_csv("Hospital_General_Information.csv")
df_medicare


,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership,...,Readmission national comparison footnote,Patient experience national comparison,Patient experience national comparison footnote,Effectiveness of care national comparison,Effectiveness of care national comparison footnote,Timeliness of care national comparison,Timeliness of care national comparison footnote,Efficient use of medical imaging national comparison,Efficient use of medical imaging national comparison footnote,Location
0,044022,CONWAY BEHAVIORAL HEALTH,2255 STURGIS ROAD,CONWAY,AR,72034,FAULKNER,(501) 205-0011,Psychiatric,Proprietary,...,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,NaN
1,054154,SAN JOSE BEHAVIORAL HEALTH,455 SILICON VALLEY BOULEVARD,SAN JOSE,CA,95138,SANTA CLARA,(669) 234-5959,Psychiatric,Proprietary,...,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,POINT (-121.77291599999998 37.246742)
2,141328,HARDIN COUNTY GENERAL HOSPITAL & CLINIC,FERRELL ROAD,ROSICLARE,IL,62982,HARDIN,(618) 285-6634,Critical Access Hospitals,Voluntary non-profit - Private,...,NaN,Not Available,16.0,Not Available,16.0,Not Available,5.0,Not Available,5.0,NaN
3,194089,GENESIS BEHAVIORAL HOSPITAL,606 LATIOLAIS ROAD,BREAUX BRIDGE,LA,70517,ST. MARTIN,(337) 442-6254,Psychiatric,Proprietary,...,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,NaN
4,194114,RIVER PLACE BEHAVIORAL HEALTH,500 RUE DE SANTE,LA PLACE,LA,70068,ST. JOHN BAPTIST,(985) 444-5100,Psychiatric,Proprietary,...,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,POINT (-90.514023 30.071081)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,180024,SPRING VIEW HOSPITAL,320 LORETTO ROAD,LEBANON,KY,40033,MARION,(270) 692-5145,Acute Care Hospitals,Proprietary,...,NaN,Same as the national average,NaN,Same as the national average,NaN,Same as the national average,NaN,Same as the national average,NaN,POINT (-85.259397 37.570325)
5316,364040,MENTAL HEALTH SERVICES FOR CLARK AND MADISON COS,474 NORTH YELLOW SPRINGS STREET,SPRINGFIELD,OH,45504,CLARK,(937) 399-9500,Psychiatric,Proprietary,...,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,POINT (-83.817429 39.930766)
5317,454088,RIO GRANDE STATE CENTER,1401 RANGERVILLE ROAD,HARLINGEN,TX,78550,CAMERON,(956) 364-8000,Psychiatric,Government - State,...,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,Not Available,19.0,POINT (-97.708011 26.16082)
5318,100276,BROWARD HEALTH CORAL SPRINGS,3000 CORAL HILLS DR,CORAL SPRINGS,FL,33065,BROWARD,(954) 344-3000,Acute Care Hospitals,Government - Hospital District or Authority,...,NaN,Below the national average,NaN,Above the national average,NaN,Same as the national average,NaN,Not Available,5.0,POINT (-80.254161 26.268779)


Determine which zip codes are in member data:

In [223]:
zip_code_list = list(df_categorical['zip_cd'].unique())


Make dataframe of member zip codes:

In [224]:
df_zc = pd.DataFrame(zip_code_list, columns = ['zip code'])

# initialize number of medicare hospitals, then reassign later
df_zc['num_MC_hosp'] = [0 for i in range(len(df_zc))]


Loop through member zip codes, counting hospitals and assigning in dataframe:

In [225]:
num_hosp_list = []

for index, row in df_zc.iterrows():

    # grab zip code
    zc_str = row[0]

    # ensure that zip code is a valid type
    try:
        zc_int = int(zc_str)
    except:
        # display any potential issues
        print('Zip code string is not numeric:', zc_str)
        continue
    
    # count the number of Medicare hospitals in each member zip code
    if zc_int in df_medicare['ZIP Code'].unique():
        
        address_rows = df_medicare.loc[df_medicare['ZIP Code'] == zc_int] # find addresses in the given zip code
        addresses = list(address_rows['Address'])

        # count hospitals
        num_hosp = len(addresses)

        # assign amount of hospitals to corresponding zip code in dataframe
        df_zc.loc[index,'num_MC_hosp'] = num_hosp
        
        num_hosp_list.append(num_hosp)


Zip code string is not numeric: other


Rename columns so we can merge on 'zip_cd' when recombining with categorical data:

In [226]:
df_zc.columns = ['zip_cd', 'num_MC_hosp']


Merging with categorical data on zip code:

In [227]:
df_categorical = df_categorical.merge(df_zc, how = 'left', on = 'zip_cd')


Creating 'hosp_per_sq_mi' feature:

In [228]:
df_categorical['hosp_per_sq_mi'] = df_categorical['num_MC_hosp'] / df_categorical['zip_sq_mi']


Extract 'hosp_per_sq_mi' from categorical data along with zip code (which we'll use to merge) in another dataframe

In [229]:
df_cat_new = df_categorical[['zip_cd', 'hosp_per_sq_mi']].copy()


Transfer feature to main df:

In [230]:
df['hosp_per_sq_mi'] = df_cat_new['hosp_per_sq_mi']


In [231]:
df.drop(columns = ['zip_cd'], inplace = True) # don't need this anymore (only used to get hosp_per_sq_mile)


Columns starting with 'pdc' have 1.1 as values that represent missing data, replace with nan

In [232]:
for col in df.columns:
    if col.startswith('pdc'):
        df[col].replace(1.1, np.nan, inplace = True)


The cleaned dataframe 'df' is now all variables with our additional 'hosp_per_sq_mi' feature.

Next, we want to bring in another dataframe (df_reduced) with only a subset of variables that we will use in the modeling process.


In [233]:
# import a manually created excel spreadsheet of the variable names that we want to maintain
vars_reduced = pd.read_excel("Vars_Reduced.xlsx")

# convert to list
var_list = list(vars_reduced['Variable'])
# can iterate through this and extract from df the variables that we want


In [234]:
# first, initialize df_reduced with IDs of each member and the target variable (0/1 for transportation issues)
# also need to add our feature that we engineered with external data, hosp_per_sq_mile
df_reduced = pd.DataFrame(df['ID'])
df_reduced['transportation_issues'] = df['transportation_issues']
df_reduced['hosp_per_sq_mi'] = df['hosp_per_sq_mi']


In [235]:
# making a list of one-hot encoded variables to compare with one-hot encoded variables in df_reduced
ohe_var_test = []

for var in var_list:
    if var in df.columns:
        continue # only want to look at columns that aren't a perfect match (which is the encoded columns)
    for col in df.columns:
        if col.startswith(var):
            ohe_var_test.append(col)


In [236]:
# adding all selected variables to df_reduced
ohe_vars = [] # tracking encoded variables

for var in var_list:
    
    # if the column name is the same, simply add it to the final dataframe
    try:
        df_reduced[var] = df[var]
        
    # categorical variables already one-hot encoded, need to account for this 
    except:
        for col in df.columns:
            if col.startswith(var):
                df_reduced[col] = df[col]
                ohe_vars.append(col)
                
assert(ohe_var_test == ohe_vars) # checking that ohe variables were all included properly


Dropping 'pdc_dep' since primarily nan values:

In [238]:
df_reduced.drop(columns=['pdc_dep'], inplace=True)

Replacing nan values with column medians:

In [239]:
for col in list(df.columns):
    try:
        df[col].replace(np.nan, df[col].median(), inplace=True)
    except:
        assert(df[col].isna().sum() == 0)
        
# making sure it worked
for col in list(df.columns):
    if df[col].isna().sum() != 0:
        print(col, df[col].isna().sum())


#### Exporting as CSV:

In [240]:
df_reduced.to_csv('Cleaned_Data.csv', index = False)
